# Task 1: AI-Powered Sales Forecasting (Starter Notebook)


This notebook provides a minimal, end-to-end baseline:
1. Load data (`date`, `sales`)
2. Explore and visualize
3. Train a simple regression-based forecaster
4. Forecast next *h* periods
5. Export `reports/forecast.csv` and plot for Power BI


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from pathlib import Path

DATA = Path("../data/raw/sample_sales.csv")  # change to your file if needed
df = pd.read_csv(DATA, parse_dates=["date"])
df = df.sort_values("date").reset_index(drop=True)
df.head()


In [ ]:

# Basic line plot of the time series
plt.figure()
plt.plot(df["date"], df["sales"])
plt.title("Sales Over Time")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.show()


In [ ]:

# Feature: convert date to an integer time index
df["t"] = (df["date"] - df["date"].min()).dt.days

# Train/test split (last 6 points as test)
h = 6
train = df.iloc[:-h].copy()
test = df.iloc[-h:].copy()

X_train = train[["t"]].values
y_train = train["sales"].values
X_test = test[["t"]].values
y_test = test["sales"].values

model = LinearRegression()
model.fit(X_train, y_train)

pred_test = model.predict(X_test)
mae = float(pd.Series(pred_test - y_test).abs().mean())
r2 = float(r2_score(y_test, pred_test))
mae, r2


In [ ]:

# Plot train/test + predictions
plt.figure()
plt.plot(train["date"], train["sales"], label="train")
plt.plot(test["date"], test["sales"], label="test")
plt.plot(test["date"], pred_test, linestyle="--", label="pred")
plt.title("Train/Test Split and Predictions")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.show()


In [ ]:

# Forecast next h periods beyond the last date
h = 6  # change horizon as needed
freq = pd.infer_freq(df["date"]) or "MS"  # default to month-start if unknown
last_date = df["date"].max()
future_dates = pd.date_range(start=last_date + pd.tseries.frequencies.to_offset(freq), periods=h, freq=freq)

# Build future design matrix (continue time index)
max_t = df["t"].max()
future_t = np.arange(max_t + (df["date"].diff().dt.days.dropna().median() or 30), 
                     max_t + (h+1)*(df["date"].diff().dt.days.dropna().median() or 30), 
                     (df["date"].diff().dt.days.dropna().median() or 30))
future_t = future_t[:h]  # ensure length h

pred_future = model.predict(future_t.reshape(-1, 1))

forecast = pd.DataFrame({"date": future_dates, "forecast": pred_future.round(0).astype(int)})
forecast.head(h)


In [ ]:

# Save artifacts
out_dir = Path("../reports")
fig_dir = out_dir / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)
forecast_path = out_dir / "forecast.csv"

forecast.to_csv(forecast_path, index=False)
forecast_path


In [ ]:

# Plot full series + forecast
plt.figure()
plt.plot(df["date"], df["sales"], label="history")
plt.plot(forecast["date"], forecast["forecast"], linestyle="--", label="forecast")
plt.title("Sales Forecast")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.savefig("../reports/figures/forecast_plot.png", bbox_inches="tight")
plt.show()
